In [1]:
import os
import random

import mlflow
import segmentation_models_pytorch as smp
from sklearn.model_selection import train_test_split
from torchvision.transforms import Compose, Resize, ToTensor, ToPILImage


from cosas.tracking import get_experiment

from cosas.paths import DATA_DIR
from cosas.data_model import COSASData, Scanncers
from cosas.datasets import COSASdataset
from cosas.misc import set_seed, train_val_split

/home/heon/anaconda3/envs/cosas/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

DEVICE = "cpu"
# experiment = get_experiment()
set_seed(42)  # TODO

cosas_data = COSASData(os.path.join(DATA_DIR, "task2"))
cosas_data.load()

(train_images, train_masks), (val_images, val_masks), (test_images, test_masks) = (
    train_val_split(cosas_data, train_val_test=(0.6, 0.2, 0.2))
)

model = smp.FPN(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    classes=1,
    activation=None,
).to(DEVICE)


In [11]:
import torch
from torch.utils.data import DataLoader

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

transform = A.Compose(
    [
        A.RandomCrop(height=224, width=224, p=1),
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]    
)


train_dataset = COSASdataset(train_images, train_masks, transform)
train_dataloader = DataLoader(train_dataset, batch_size=4)
num_epochs = 2
n_steps = len(train_dataloader)
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    
    for step, (images, masks) in enumerate(train_dataloader):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE).unsqueeze(1).float()

        # Forward pass
        outputs = model(images)
        
        # Compute loss
        loss = loss_fn(outputs, masks)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Step({step}/{n_steps}): Loss: {loss}")

Epoch 1/2, Step(0/18): Loss: 0.5385966300964355
Epoch 1/2, Step(1/18): Loss: 0.7404068112373352
Epoch 1/2, Step(2/18): Loss: 0.4401918053627014
Epoch 1/2, Step(3/18): Loss: 0.7889130115509033
Epoch 1/2, Step(4/18): Loss: 0.4913847744464874
Epoch 1/2, Step(5/18): Loss: 0.38508084416389465
Epoch 1/2, Step(6/18): Loss: 1.963456392288208
Epoch 1/2, Step(7/18): Loss: 1.784023404121399
Epoch 1/2, Step(8/18): Loss: 0.6622202396392822
Epoch 1/2, Step(9/18): Loss: 0.24942857027053833
Epoch 1/2, Step(10/18): Loss: 0.4482574760913849
Epoch 1/2, Step(11/18): Loss: 0.816438615322113
Epoch 1/2, Step(12/18): Loss: 0.605463981628418
Epoch 1/2, Step(13/18): Loss: 0.22888633608818054
Epoch 1/2, Step(14/18): Loss: 0.8708387017250061
Epoch 1/2, Step(15/18): Loss: 0.3965553641319275
Epoch 1/2, Step(16/18): Loss: 0.2841457426548004
Epoch 1/2, Step(17/18): Loss: 0.5831342935562134
Epoch 2/2, Step(0/18): Loss: 0.30878451466560364
Epoch 2/2, Step(1/18): Loss: 0.2993624210357666
Epoch 2/2, Step(2/18): Loss: 0.9